# __Search image by keyword__

## Preface

- Tutorial Difficulty : ★☆☆☆☆
- 10 min read
- Languages : [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- File location : tutorial_en/thanosql_search/search_image_by_keyword.ipynb
- References : [Food Image and Nutrition Text Introduction Dataset](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=74)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Keyword-Image Search Service</h4>
    <p>ThanoSQL provides a search function that allows you to return images to results through keywords. Use an image classification model, etc. to set the keyword desired by the user to the target value, and add a column that indexes the updated image using the learned model. If the text-image search feature analyzes meaning from images and provides similar images, keyword-image search finds the image that corresponds to the desired target value (category). </p>
</div>

Search has the dictionary meaning of "finding the necessary materials in a book or computer according to its purpose." ThanoSQL keyword-image search works a little differently than searching for information within a DB with the inclusion of a specific word (keyword). Keyword-based image search creates a model that pre-learns and predicts words from the features of an image, and provides the image with the highest probability of being included in a specific keyword through that model.

**Below is an example and utilization of the ThanoSQL keyword image search algorithm.**

- Use the categories of shopping categories as learning data to create a learning model, and use them to create an index column in an existing/new image. The index column combines numerical attributes such as image registration dates to provide more sophisticated searches.
- You can create a My image search service by utilizing various results such as similar image search results and text-image search results covered in the following tutorial.

<div class="admonition note">
    <h4 class="admonition-title"> In this tutorial</h4>
    <p>👉 Use a combination of the "<strong>SEARCH</strong>" query syntax provided by ThanoSQL and the "<strong>SELECT</strong>" query syntax provided by traditional PostgreSQL to perform the desired image search using specific keywords. </p>
</div>

<div class="admonition tip">
    <h4 class="admonition-title">Dataset Description</h4>
    <p><code>Introduction to Food Images and Nutrition Information Text</code> dataset is organized by the Ministry of Science and ICT and supported by the Korea Intelligence Information Society Agency, and consists of 400 Korean Davido eating out menus and Korean food menus. It consists of 842,000 images, and this tutorial uses only a few (10 types, 1,190 photos) in that dataset. </p>
</div>

## __0. Prepare Dataset__

To use the query syntax of ThanoSQL, you must create an API token and run the query below, as mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace).

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [ ]:
%%thanosql
GET THANOSQL DATASET diet_data
OPTIONS (overwrite=True)

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Use the query syntax "<strong>GET THANOSQL DATASET</strong>" to save the desired dataset to the workspace. </li>
        <li>Specifies the options to use for <strong>GET THANOSQL DATASET</strong> via the query syntax "<strong>OPTIONS</strong>".
        <ul>
            <li>"overwrite" : Set whether to overwrite if a dataset with the same name exists. If True, the old dataset is replaced with the new dataset (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

In [ ]:
%%thanosql
COPY diet 
OPTIONS (overwrite=True)
FROM "thanosql-dataset/diet_data/diet.csv"

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Use the "<strong>COPY</strong>" query syntax to specify the dataset name to be saved in the DB. </li>
        <li>Specifies the options to use for <strong>COPY</strong> via the query syntax "<strong>OPTIONS</strong>".
        <ul>
            <li>"overwrite" : Set whether or not overwrite is possible if a dataset with the same name exists in the DB. If True, the old dataset is replaced with the new dataset (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a keyword-image search model, we use the <mark style="background-color:#FFEC92">diet</mark> table stored in ThanoSQL DB. Execute the query syntax below and check the contents of the table.

In [ ]:
%%thanosql
SELECT * 
FROM diet

<div class="admonition note">
    <h4 class="admonition-title">Understanding the Data Table</h4>
    <p>The <mark style="background-color:#FFEC92">diet</mark> table contains the following information. </p>
    <ul>
        <li><mark style="background-color:#D7D0FF">image_path</mark> : image path </li>
        <li><mark style="background-color:#D7D0FF">label</mark> : filename</li>
    </ul>
</div>

## __2. Create Keyword Search Model__

In order to search for images, it is necessary to learn the existing data table and create criteria for future searches. To do this, we create an image classification model using the dataset we identified in the previous step. Execute the query syntax below to create a model named <mark style="background-color:#E9D7FD ">diet_image_classification</mark>.  
(Estimated duration of query execution: 3 min)

In [ ]:
%%thanosql
BUILD MODEL diet_image_classification
USING ConvNeXt_Tiny
OPTIONS (
    image_col='image_path', 
    label_col='label', 
    epochs=1,
    overwrite=True
    )
AS 
SELECT *
FROM diet

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Create and train the model <mark style="background-color:#E9D7FD ">diet_image_classification</mark> using the query syntax "<strong>BUILD MODEL</strong>".</li>
        <li>Specify to use <code>ConvNeXt_Tiny</code> as the base model through the query syntax "<strong>USING</strong>".</li>
        <li>"<strong>OPTIONS</strong>" Specifies the options to use for model creation via the query syntax.
        <ul>
            <li>"image_col" : The name of the column containing the image path.</li>
            <li>"label_col" : The name of the column containing the target value information</li>
            <li>"epochs" : number of times to train all training datasets</li>
            <li>"overwrite" : Set whether overwriting is possible if a model with the same name exists. If True, the old model is changed to the new model (True|False, DEFAULT : False)</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Use the generated model to check the keyword-image search model__

Use the image prediction model created in the previous step(<mark style="background-color:#E9D7FD ">diet_image_classification</mark>) to predict the target value of a specific image. After executing the query below, the prediction result is stored and returned in the <mark style="background-color:#D7D0FF">predicted</mark> column.

In [ ]:
%%thanosql
PREDICT USING diet_image_classification
AS 
SELECT *
FROM diet

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Use the <mark style="background-color:#E9D7FD ">diet_image_classification</mark> model created in the previous step for prediction via the query syntax "<strong>PREDICT USING</strong>".</li>
    </ul>
</div>

## __4. Search using the generated model__

Now use the "__PREDICT USING__", "__SELECT__", "__WHERE__" query syntax to retrieve only data with a specific condition. You can search only data for which <mark style="background-color:#E9D7FD ">label</mark> is '사과파이'('apple pie') and the prediction result is also '사과파이' and write the query syntax as follows.

In [ ]:
%%thanosql
SELECT A.image_path, A.label, B.predicted 
FROM diet A
LEFT JOIN (
    SELECT * 
    FROM (PREDICT USING diet_image_classification 
    AS SELECT * FROM diet)) B 
ON A.image_path = B.image_path
WHERE A.label = B.predicted
AND A.label LIKE '사과파이'
LIMIT 10

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SELECT * FROM (...)</strong>" query syntax to select all the results of the query syntax starting with "<strong>PREDICT USING</strong>".</li>
        <li>Set the condition through the query syntax "<strong>WHERE</strong>". This condition is followed by "<strong>AND</strong>".
        <ul>
            <li>"label = predicted" : Extracts only data with the same value in <mark style="background-color:#D7D0FF ">label</mark> column and <mark style="background-color:#D7D0FF ">predicted</mark> column.</li>
            <li>"label = '사과파이'" : <mark style="background-color:#D7D0FF ">label</mark> Extracts only data whose column is 'apple pie'.</li>
        </ul>
        </li>
    </ul>
</div>

## **5. In Conclusion**

In this tutorial, we constructed and utilized a model to search for food images related to keywords using the `food image dataset`. In this tutorial, we focused on operation rather than accuracy. You can improve the accuracy of your model by adjusting various options, such as the number of lessons learned and the number of data in the build option. Followed by similar image search models, image search tutorials using text descriptions, and create your own various search services.

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties in creating your own model using ThanoSQL or applying it to the service, please feel free to contact us below😊</p>
    <p>For inquiries regarding keyword-image search model building: contact@smartmind.team</p>
</div>